In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
### Extract feature ###

import os
import numpy as np
import pandas as pd
from scipy.stats import skew, kurtosis
from scipy.signal import welch
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
import json

def extract_time_features(signal, threshold=0.01):
    mav = np.mean(np.abs(signal))
    rms = np.sqrt(np.mean(signal ** 2))
    var = np.var(signal)
    zc = np.sum(np.diff(np.sign(signal)) != 0)
    ssc = np.sum(np.diff(np.sign(np.diff(signal))) != 0)
    wl = np.sum(np.abs(np.diff(signal)))
    wamp = np.sum(np.abs(np.diff(signal)) > threshold)
    skewness = skew(signal)
    kurt = kurtosis(signal)
    return [mav, rms, var, zc, ssc, wl, wamp, skewness, kurt]

def extract_freq_features(signal, fs=1000):
    f, pxx = welch(signal, fs=fs)
    mnf = np.sum(f * pxx) / np.sum(pxx)
    mdf = f[np.cumsum(pxx) >= np.sum(pxx) / 2][0]
    m0 = np.sum(pxx)
    m2 = np.sum((f ** 2) * pxx)
    m4 = np.sum((f ** 4) * pxx)
    return [mnf, mdf, m0, m2, m4]

def extract_features_from_window(window, fs=1000):
    features = []
    for ch in range(window.shape[1]):
        sig = window[:, ch]
        features += extract_time_features(sig)
        features += extract_freq_features(sig, fs)
    return features

def create_windows(data, window_size, stride):
    return [data[i:i+window_size] for i in range(0, len(data) - window_size + 1, stride) if i+window_size <= len(data)]

# Parameters
window_size = 200
stride = 50
fs = 1000

feature_records = []
# Data Loading Loop
for subj in range(1, 28):  # S1 ~ S9
    subject_id = f"S{subj}_A1_E1"
    for label in range(1, 13):  # label_1 ~ label_12
        for rep in range(1, 11):  # rep_1 ~ rep_10
            print(f"Processing {subject_id}, label {label}, rep {rep}")
            file_path = f"./drive/MyDrive/preprocessed_emg_by_label/{subject_id}/label_{label}/rep_{rep}.csv"
            if not os.path.exists(file_path):
                print("missing:", file_path)
                continue
            try:
                df = pd.read_csv(file_path)
                data = df.values.astype(np.float32)
                if data.shape[1] < 2:
                    continue
                windows = create_windows(data, window_size, stride)
                for window in windows:
                    feats = extract_features_from_window(window, fs)
                    feats = [float(x) for x in feats]
                    record = {
                        "subject": subj,
                        "label": label,
                        "rep": rep,
                        "features": feats
                    }
                    feature_records.append(record)
            except Exception as e:
                print(f"⚠️ Error in {file_path}: {e}")

output_path = "./drive/MyDrive/experiment_data/extracted_emg_features.json"
with open(output_path, "w") as f:
    json.dump(feature_records, f)

Processing S1_A1_E1, label 1, rep 1


/usr/local/lib/python3.11/dist-packages/scipy/signal/_spectral_py.py:790: UserWarning: nperseg = 256 is greater than input length  = 200, using nperseg = 200
  freqs, _, Pxy = _spectral_helper(x, y, fs, window, nperseg, noverlap,
/usr/local/lib/python3.11/dist-packages/scipy/signal/_spectral_py.py:790: UserWarning: nperseg = 256 is greater than input length  = 200, using nperseg = 200
  freqs, _, Pxy = _spectral_helper(x, y, fs, window, nperseg, noverlap,


Processing S1_A1_E1, label 1, rep 2
Processing S1_A1_E1, label 1, rep 3
Processing S1_A1_E1, label 1, rep 4


/usr/local/lib/python3.11/dist-packages/scipy/signal/_spectral_py.py:790: UserWarning: nperseg = 256 is greater than input length  = 200, using nperseg = 200
  freqs, _, Pxy = _spectral_helper(x, y, fs, window, nperseg, noverlap,
/usr/local/lib/python3.11/dist-packages/scipy/signal/_spectral_py.py:790: UserWarning: nperseg = 256 is greater than input length  = 200, using nperseg = 200
  freqs, _, Pxy = _spectral_helper(x, y, fs, window, nperseg, noverlap,


Processing S1_A1_E1, label 1, rep 5
Processing S1_A1_E1, label 1, rep 6


...

Processing S27_A1_E1, label 12, rep 4
Processing S27_A1_E1, label 12, rep 5
Processing S27_A1_E1, label 12, rep 6


/usr/local/lib/python3.11/dist-packages/scipy/signal/_spectral_py.py:790: UserWarning: nperseg = 256 is greater than input length  = 200, using nperseg = 200
  freqs, _, Pxy = _spectral_helper(x, y, fs, window, nperseg, noverlap,
/usr/local/lib/python3.11/dist-packages/scipy/signal/_spectral_py.py:790: UserWarning: nperseg = 256 is greater than input length  = 200, using nperseg = 200
  freqs, _, Pxy = _spectral_helper(x, y, fs, window, nperseg, noverlap,
/usr/local/lib/python3.11/dist-packages/scipy/signal/_spectral_py.py:790: UserWarning: nperseg = 256 is greater than input length  = 200, using nperseg = 200
  freqs, _, Pxy = _spectral_helper(x, y, fs, window, nperseg, noverlap,


Processing S27_A1_E1, label 12, rep 7
Processing S27_A1_E1, label 12, rep 8
Processing S27_A1_E1, label 12, rep 9


/usr/local/lib/python3.11/dist-packages/scipy/signal/_spectral_py.py:790: UserWarning: nperseg = 256 is greater than input length  = 200, using nperseg = 200
  freqs, _, Pxy = _spectral_helper(x, y, fs, window, nperseg, noverlap,
/usr/local/lib/python3.11/dist-packages/scipy/signal/_spectral_py.py:790: UserWarning: nperseg = 256 is greater than input length  = 200, using nperseg = 200
  freqs, _, Pxy = _spectral_helper(x, y, fs, window, nperseg, noverlap,
/usr/local/lib/python3.11/dist-packages/scipy/signal/_spectral_py.py:790: UserWarning: nperseg = 256 is greater than input length  = 200, using nperseg = 200
  freqs, _, Pxy = _spectral_helper(x, y, fs, window, nperseg, noverlap,
/usr/local/lib/python3.11/dist-packages/scipy/signal/_spectral_py.py:790: UserWarning: nperseg = 256 is greater than input length  = 200, using nperseg = 200
  freqs, _, Pxy = _spectral_helper(x, y, fs, window, nperseg, noverlap,


Processing S27_A1_E1, label 12, rep 10


In [5]:
### baseline ###
import json
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

# upload JSON data
with open("./drive/MyDrive/experiment_data/extracted_emg_features.json", "r") as f:
    feature_records = json.load(f)

# split train/test
train_records = [r for r in feature_records if r["subject"] <= 22]
test_records  = [r for r in feature_records if r["subject"] > 22]

# extract feature and label
def extract_data(record_list):
    X = [r["features"] for r in record_list]
    y = [r["label"] - 1 for r in record_list]  # 0-indexed
    return np.array(X), np.array(y)

X_train, y_train = extract_data(train_records)
X_test,  y_test = extract_data(test_records)

# scaling
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled  = scaler.transform(X_test)

clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train_scaled, y_train)

# model predict
y_pred = clf.predict(X_test_scaled)
print("\n Subject-independent Test Result:")
print(classification_report(y_test, y_pred))


 Subject-independent Test Result:
              precision    recall  f1-score   support

           0       0.28      0.31      0.30       203
           1       0.48      0.43      0.45       222
           2       0.21      0.21      0.21       322
           3       0.53      0.14      0.22       224
           4       0.43      0.44      0.43       170
           5       0.46      0.46      0.46       227
           6       0.11      0.12      0.11       189
           7       0.31      0.29      0.30       246
           8       0.25      0.12      0.16       233
           9       0.38      0.60      0.47       198
          10       0.32      0.48      0.38       161
          11       0.12      0.17      0.14       205

    accuracy                           0.30      2600
   macro avg       0.32      0.31      0.30      2600
weighted avg       0.32      0.30      0.30      2600



In [6]:
### Applying VAE ###
import json
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.preprocessing import StandardScaler

# upload JSON data
with open("./drive/MyDrive/experiment_data/extracted_emg_features.json", "r") as f:
    feature_records = json.load(f)

X = np.array([rec["features"] for rec in feature_records], dtype=np.float32)

X = np.nan_to_num(X, nan=0.0, posinf=0.0, neginf=0.0)

# Scaling
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

input_dim = X_scaled.shape[1]
latent_dim = 32

# Sampling Layer
class Sampling(layers.Layer):
    def call(self, inputs):
        z_mean, z_log_var = inputs
        epsilon = tf.random.normal(shape=tf.shape(z_mean))
        return z_mean + tf.exp(0.5 * z_log_var) * epsilon

# Encoder
encoder_inputs = keras.Input(shape=(input_dim,))
x = layers.Dense(128, activation="relu")(encoder_inputs)
x = layers.Dense(64, activation="relu")(x)
z_mean = layers.Dense(latent_dim, name="z_mean")(x)
z_log_var = layers.Dense(latent_dim, name="z_log_var")(x)
z = Sampling()([z_mean, z_log_var])
encoder = keras.Model(encoder_inputs, [z_mean, z_log_var, z], name="encoder")

# Decoder
latent_inputs = keras.Input(shape=(latent_dim,))
x = layers.Dense(64, activation="relu")(latent_inputs)
x = layers.Dense(128, activation="relu")(x)
decoder_outputs = layers.Dense(input_dim)(x)  # Linear output
decoder = keras.Model(latent_inputs, decoder_outputs, name="decoder")

# VAE Definition
class VAE(keras.Model):
    def __init__(self, encoder, decoder, **kwargs):
        super().__init__(**kwargs)
        self.encoder = encoder
        self.decoder = decoder
        self.total_loss_tracker = keras.metrics.Mean(name="total_loss")
        self.reconstruction_loss_tracker = keras.metrics.Mean(name="reconstruction_loss")
        self.kl_loss_tracker = keras.metrics.Mean(name="kl_loss")

    @property
    def metrics(self):
        return [self.total_loss_tracker, self.reconstruction_loss_tracker, self.kl_loss_tracker]

    def train_step(self, data):
        if isinstance(data, tuple): data = data[0]
        with tf.GradientTape() as tape:
            z_mean, z_log_var, z = self.encoder(data)
            reconstruction = self.decoder(z)
            recon_loss = tf.reduce_mean(tf.reduce_sum(tf.square(data - reconstruction), axis=1))
            z_log_var_clipped = tf.clip_by_value(z_log_var, -10.0, 10.0)
            kl_loss = -0.5 * tf.reduce_mean(tf.reduce_sum(
                1 + z_log_var_clipped - tf.square(z_mean) - tf.exp(z_log_var_clipped), axis=1))
            total_loss = recon_loss + kl_loss
        grads = tape.gradient(total_loss, self.trainable_weights)
        self.optimizer.apply_gradients(zip(grads, self.trainable_weights))
        self.total_loss_tracker.update_state(total_loss)
        self.reconstruction_loss_tracker.update_state(recon_loss)
        self.kl_loss_tracker.update_state(kl_loss)
        return {
            "loss": self.total_loss_tracker.result(),
            "reconstruction_loss": self.reconstruction_loss_tracker.result(),
            "kl_loss": self.kl_loss_tracker.result(),
        }

# training
vae = VAE(encoder, decoder)
vae.compile(optimizer=keras.optimizers.Adam(learning_rate=1e-4))
vae.fit(X_scaled, epochs=100, batch_size=128)

# extract latent and save
z_mean_vals, _, _ = encoder.predict(X_scaled)

for i, rec in enumerate(feature_records):
    rec["latent"] = z_mean_vals[i].tolist()

output_path = "./drive/MyDrive/experiment_data/vae_latent_features.json"

with open(output_path, "w") as f:
    json.dump(feature_records, f, indent=2)

print(f"Latent vectors saved: {output_path}")


Epoch 1/100
97/97 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - kl_loss: 9.0550 - loss: 153.0191 - reconstruction_loss: 143.9642
Epoch 2/100
97/97 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - kl_loss: 4.7237 - loss: 140.4025 - reconstruction_loss: 135.6788
Epoch 3/100
97/97 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - kl_loss: 3.5660 - loss: 136.1327 - reconstruction_loss: 132.5667
Epoch 4/100
97/97 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - kl_loss: 4.3005 - loss: 130.5741 - reconstruction_loss: 126.2736
Epoch 5/100
97/97 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - kl_loss: 6.2545 - loss: 120.9708 - reconstruction_loss: 114.7163
Epoch 6/100
97/97 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - kl_loss: 6.4907 - loss: 112.5741 - reconstruction_loss: 106.0834
Epoch 7/100
97/97 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - kl_loss: 7.3588 - loss: 109.3329 - reconstruction_loss: 101.9742
Epoch 8/100
97/97 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - kl_loss: 8.0667 - loss: 104.2677 - reconstruction_loss: 96.2010
Epoch 9/100
97/97 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - kl_l

In [7]:
### k-means ###
import json
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans

# upload JSON data
json_path = "./drive/MyDrive/experiment_data/vae_latent_features.json"

with open(json_path, "r") as f:
    feature_records = json.load(f)

# extract latent and label
X = np.array([rec["latent"] for rec in feature_records], dtype=np.float32)
y = np.array([rec["label"] - 1 for rec in feature_records])  # 0-indexed

# scaling
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# clustering for each label
n_clusters_per_class = 10
cluster_results = []

for label_id in np.unique(y):
    mask = (y == label_id)
    X_label = X_scaled[mask]

    # kmeans
    kmeans = KMeans(n_clusters=n_clusters_per_class, random_state=0, n_init='auto')
    cluster_ids = kmeans.fit_predict(X_label)
    centers = kmeans.cluster_centers_

    # calculate distance
    dists = np.linalg.norm(X_label - centers[cluster_ids], axis=1)
    sigma = np.mean(dists) or 1.0

    label_indices = np.where(mask)[0]
    for i, idx in enumerate(label_indices):
        record = feature_records[idx]
        record["cluster_id"] = int(cluster_ids[i])
        record["distance"] = float(dists[i])
        norm_d = dists[i] / sigma
        record["weight"] = float(1 / (1 + np.exp(20 * (norm_d - 1.0))))  # calculate weight
        cluster_results.append(record)

# save
output_path = "./drive/MyDrive/experiment_data/clustered_kmeans_vae_latent_features.json"
with open(output_path, "w") as f:
    json.dump(cluster_results, f, indent=2)
print(f"ouput saved: {output_path}")


ouput saved: ./drive/MyDrive/experiment_data/clustered_kmeans_vae_latent_features.json


In [8]:
### GMM ###
import json
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.mixture import GaussianMixture

# upload JSON data
json_path = "./drive/MyDrive/experiment_data/vae_latent_features.json"

with open(json_path, "r") as f:
    feature_records = json.load(f)

# extract latent and label
X = np.array([rec["latent"] for rec in feature_records], dtype=np.float32)
y = np.array([rec["label"] - 1 for rec in feature_records])  # 0-indexed

# scaling
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# clustering for each label
n_clusters_per_class = 10
cluster_results = []

for label_id in np.unique(y):
    mask = (y == label_id)
    X_label = X_scaled[mask]

    # GMM
    gmm = GaussianMixture(n_components=n_clusters_per_class, random_state=0)
    cluster_ids = gmm.fit_predict(X_label)
    centers = gmm.means_

    # calculate distance
    dists = np.linalg.norm(X_label - centers[cluster_ids], axis=1)
    sigma = np.mean(dists) or 1.0

    label_indices = np.where(mask)[0]
    for i, idx in enumerate(label_indices):
        record = feature_records[idx]
        record["cluster_id"] = int(cluster_ids[i])
        record["distance"] = float(dists[i])
        norm_d = dists[i] / sigma
        record["weight"] = float(1 / (1 + np.exp(20 * (norm_d - 1.0))))  # calculate weight
        cluster_results.append(record)

# save
output_path = "./drive/MyDrive/experiment_data/clustered_gmm_vae_latent_features.json"
with open(output_path, "w") as f:
    json.dump(cluster_results, f, indent=2)
print(f"output saved: {output_path}")


output saved: ./drive/MyDrive/experiment_data/clustered_gmm_vae_latent_features.json


In [9]:
### HDBSCAN ###
import json
import numpy as np
from sklearn.preprocessing import StandardScaler
import hdbscan

# upload JSON data
json_path = "./drive/MyDrive/experiment_data/vae_latent_features.json"

with open(json_path, "r") as f:
    feature_records = json.load(f)

# extract latent and label
X = np.array([rec["latent"] for rec in feature_records], dtype=np.float32)
y = np.array([rec["label"] - 1 for rec in feature_records])  # 0-indexed

# scaling
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# clustering for each label
cluster_results = []

for label_id in np.unique(y):
    mask = (y == label_id)
    X_label = X_scaled[mask]

    # HDBSCAN
    hdb = hdbscan.HDBSCAN(min_cluster_size=10, min_samples=5, prediction_data=True)
    cluster_ids = hdb.fit_predict(X_label)
    soft_probs = hdb.probabilities_

    label_indices = np.where(mask)[0]
    for i, idx in enumerate(label_indices):
        record = feature_records[idx]
        record["cluster_id"] = int(cluster_ids[i])
        record["weight"] = float(soft_probs[i])     # calculate weight
        cluster_results.append(record)

# save
output_path = "./drive/MyDrive/experiment_data/clustered_hdbscan_vae_latent_features.json"
with open(output_path, "w") as f:
    json.dump(cluster_results, f, indent=2)
print(f"output saved: {output_path}")



/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11

output saved: ./drive/MyDrive/experiment_data/clustered_hdbscan_vae_latent_features.json


In [10]:
### examine kmeans distribution ###

import json
import numpy as np
from collections import defaultdict, Counter

# upload JSON data
input_path = "./drive/MyDrive/experiment_data/clustered_kmeans_vae_latent_features.json"
output_path = "./drive/MyDrive/experiment_data/filtered_clustered_kmeans_vae_latent_features.json"

with open(input_path, "r") as f:
    data = json.load(f)

# Separate by label
records_by_label = defaultdict(list)
for rec in data:
    records_by_label[rec["label"]].append(rec)

filtered_records = []

# Check distribution and filter by label
for label, recs in records_by_label.items():
    cluster_ids = [r["cluster_id"] for r in recs]
    counts = np.bincount(cluster_ids)

    mean = np.mean(counts)
    std = np.std(counts)

    if std / mean < 0.1:
        filtered_records.extend(recs)
    else:
        threshold = int(0.8 * len(recs))

        order = np.argsort(counts)[::-1]
        cumsum = np.cumsum(counts[order])
        k_keep = np.where(cumsum >= threshold)[0][0]
        clusters_to_keep = order[:k_keep + 1]

        filtered = [r for r in recs if r["cluster_id"] in clusters_to_keep]
        filtered_records.extend(filtered)

# save
with open(output_path, "w") as f:
    json.dump(filtered_records, f, indent=2)
print(f"output saved: {output_path}")


output saved: ./drive/MyDrive/experiment_data/filtered_clustered_kmeans_vae_latent_features.json


In [11]:
### examine gmm distribution ###

import json
import numpy as np
from collections import defaultdict, Counter

# upload JSON data
input_path = "./drive/MyDrive/experiment_data/clustered_gmm_vae_latent_features.json"
output_path = "./drive/MyDrive/experiment_data/filtered_clustered_gmm_vae_latent_features.json"

with open(input_path, "r") as f:
    data = json.load(f)

# Separate by label
records_by_label = defaultdict(list)
for rec in data:
    records_by_label[rec["label"]].append(rec)

filtered_records = []

# Check distribution and filter by label
for label, recs in records_by_label.items():
    cluster_ids = [r["cluster_id"] for r in recs]
    counts = np.bincount(cluster_ids)

    mean = np.mean(counts)
    std = np.std(counts)

    if std / mean < 0.1:
        filtered_records.extend(recs)
    else:
        threshold = int(0.8 * len(recs))

        order = np.argsort(counts)[::-1]
        cumsum = np.cumsum(counts[order])
        k_keep = np.where(cumsum >= threshold)[0][0]
        clusters_to_keep = order[:k_keep + 1]

        filtered = [r for r in recs if r["cluster_id"] in clusters_to_keep]
        filtered_records.extend(filtered)

# save
with open(output_path, "w") as f:
    json.dump(filtered_records, f, indent=2)
print(f"output saved: {output_path}")


output saved: ./drive/MyDrive/experiment_data/filtered_clustered_gmm_vae_latent_features.json


In [12]:
### remove outlier of hdbscan ###

import json

# upload JSON data
input_path = "./drive/MyDrive/experiment_data/clustered_hdbscan_vae_latent_features.json"
output_path = "./drive/MyDrive/experiment_data/filtered_clustered_hdbscan_vae_latent_features.json"

with open(input_path, "r") as f:
    data = json.load(f)

# remove cluster_id == -1
filtered_data = [rec for rec in data if rec["cluster_id"] != -1]


# save
with open(output_path, "w") as f:
    json.dump(filtered_data, f, indent=2)
print(f"output saved: {output_path}")


output saved: ./drive/MyDrive/experiment_data/filtered_clustered_hdbscan_vae_latent_features.json


In [13]:
### classification of kmeans data ###
import json
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

def extract_data(record_list):
    X = [r["features"] for r in record_list]
    y = [r["label"] - 1 for r in record_list]
    return np.array(X), np.array(y)

# upload train data
with open("./drive/MyDrive/experiment_data/filtered_clustered_kmeans_vae_latent_features.json", "r") as f:
    train_all_kmeans = json.load(f)

train_records_kmeans = [r for r in train_all_kmeans if r["subject"] <= 22]
X_train_kmeans, y_train_kmeans = extract_data(train_records_kmeans)

# scaling
scaler = StandardScaler()
X_train_scaled_kmeans = scaler.fit_transform(X_train_kmeans)

# train model
clf_kmeans = RandomForestClassifier(n_estimators=100, random_state=42)
clf_kmeans.fit(X_train_scaled_kmeans, y_train_kmeans)

RandomForestClassifier(random_state=42)

In [14]:
### classification of gmm data ###

# upload train data
with open("./drive/MyDrive/experiment_data/filtered_clustered_gmm_vae_latent_features.json", "r") as f:
    train_all_gmm = json.load(f)

train_records_gmm = [r for r in train_all_gmm if r["subject"] <= 22]
X_train_gmm, y_train_gmm = extract_data(train_records_gmm)

# scaling
scaler = StandardScaler()
X_train_scaled_gmm = scaler.fit_transform(X_train_gmm)

# train model
clf_gmm = RandomForestClassifier(n_estimators=100, random_state=42)
clf_gmm.fit(X_train_scaled_gmm, y_train_gmm)

RandomForestClassifier(random_state=42)

In [16]:
### classification of hdbscan data ###

# upload train data
with open("./drive/MyDrive/experiment_data/filtered_clustered_hdbscan_vae_latent_features.json", "r") as f:
    train_all_hdbscan = json.load(f)

train_records_hdbscan = [r for r in train_all_hdbscan if r["subject"] <= 22]
X_train_hdbscan, y_train_hdbscan = extract_data(train_records_hdbscan)

# scaling
scaler = StandardScaler()
X_train_scaled_hdbscan = scaler.fit_transform(X_train_hdbscan)

# train model
clf_hdbscan = RandomForestClassifier(n_estimators=100, random_state=42)
clf_hdbscan.fit(X_train_scaled_hdbscan, y_train_hdbscan)

RandomForestClassifier(random_state=42)

In [17]:
with open("./drive/MyDrive/experiment_data/clustered_kmeans_vae_latent_features.json", "r") as f:
    test_all_kmeans = json.load(f)

test_records_kmeans  = [r for r in test_all_kmeans  if r["subject"] >= 22]
X_test_kmeans,  y_test_kmeans  = extract_data(test_records_kmeans)
X_test_scaled_kmeans  = scaler.transform(X_test_kmeans)

with open("./drive/MyDrive/experiment_data/clustered_gmm_vae_latent_features.json", "r") as f:
    test_all_gmm = json.load(f)

test_records_gmm  = [r for r in test_all_gmm  if r["subject"] >= 22]
X_test_gmm,  y_test_gmm  = extract_data(test_records_gmm)
X_test_scaled_gmm  = scaler.transform(X_test_gmm)

with open("./drive/MyDrive/experiment_data/clustered_hdbscan_vae_latent_features.json", "r") as f:
    test_all_hdbscan = json.load(f)

test_records_hdbscan  = [r for r in test_all_hdbscan  if r["subject"] >= 22]
X_test_hdbscan,  y_test_hdbscan  = extract_data(test_records_hdbscan)
X_test_scaled_hdbscan  = scaler.transform(X_test_hdbscan)

In [18]:
y_pred_kmeans = clf_kmeans.predict(X_test_scaled_kmeans)
print("\n kmeans Test Result:")
print(classification_report(y_test_kmeans, y_pred_kmeans))

y_pred_gmm = clf_gmm.predict(X_test_scaled_gmm)
print("\n gmm Test Result:")
print(classification_report(y_test_gmm, y_pred_gmm))

y_pred_hdbscan = clf_hdbscan.predict(X_test_scaled_hdbscan)
print("\n hdbscan Test Result:")
print(classification_report(y_test_hdbscan, y_pred_hdbscan))


 kmeans Test Result:
              precision    recall  f1-score   support

           0       0.49      0.44      0.47       263
           1       0.72      0.64      0.68       298
           2       0.45      0.45      0.45       393
           3       0.79      0.26      0.39       282
           4       0.61      0.61      0.61       216
           5       0.53      0.56      0.54       268
           6       0.32      0.30      0.31       235
           7       0.38      0.39      0.38       279
           8       0.48      0.24      0.32       285
           9       0.28      0.71      0.40       239
          10       0.53      0.42      0.47       193
          11       0.26      0.31      0.28       268

    accuracy                           0.44      3219
   macro avg       0.49      0.44      0.44      3219
weighted avg       0.49      0.44      0.44      3219


 gmm Test Result:
              precision    recall  f1-score   support

           0       0.54      0.41    